In [21]:
import os
import re
import json
from tqdm import tqdm

from utils import DATA_ROOT
from utils import load_jsonl_iteratively

response_fn = os.path.join(DATA_ROOT, "response", "en_pair", "qa-gen.response.s0e620000.jsonl")
datasets_fn = os.path.join(DATA_ROOT, "datasets/medical", "en_pair", "data.jsonl")

In [22]:
items = {}
for doc_item in tqdm(load_jsonl_iteratively(datasets_fn), desc=f"Reading documents from preprocessed data: {datasets_fn}"):
    items[str(doc_item['docid'])] = doc_item

Reading documents from preprocessed data: /data/xzhao/dataset/roman-pretrain/datasets/medical/en_pair/data.jsonl: 614444it [00:07, 77413.30it/s] 


In [23]:
non_match_doc, less_5_qa = [], []
non_error_docs, non_match_text = [], []
# qaffix_affix = r"#+\s*(?:\d+\.\s*)?(?:question|Question)(?:\d+)?(?:\s*\d+)?(?::|：)"
qaffix_affix = r'(?:#+\s*)?(?:\*\*)?(?:question|Question)(?:\d+)?(?:\*\*)?(?:\s*\d+)?\s*(?::|：)\s*'
qaffix_pattern = qaffix_affix + r"(?!\s*\.)"

for i, res_item in tqdm(enumerate(load_jsonl_iteratively(response_fn)), desc="Processing QA response"):
    assert str(res_item['id'][9:]) in items
    generation = res_item['choices'][0]['message']['content']
    if "</think>" in generation:
        qa_text = generation.split("</think>")[1].strip()
    elif "\n\n\n" in generation and len(generation.split("\n\n\n")) == 2:
        qa_text = generation.split("\n\n\n")[1].strip()
    elif sum(1 for _ in re.finditer(qaffix_pattern, generation)) >= 5:
        qa_text = generation[list(re.finditer(qaffix_pattern, generation))[0].start():]
    elif sum(1 for _ in re.finditer(qaffix_pattern, generation)) >= 5:
        qa_text = generation[list(re.finditer(qaffix_pattern, generation))[0].start():]
    else:
        non_match_doc.append(generation)
        continue

    start_indices = []
    for match in re.finditer(qaffix_pattern, qa_text):
        start_indices.append(match.start())
    start_indices.append(-1)
    
    if len(start_indices) <= 2:
        less_5_qa.append((start_indices, qa_text))
        continue
    
    # aaffix = r'#+\s*(?:\d+\.\s*)?(?:Answer|answer)(?:\d+)?(?:\s*\d+)?(?::|：)\s*'
    aaffix = r'(?:#+\s*)?(?:\*\*)?(?:Answer|answer)(?:\d+)?(?:\*\*)?(?:\s*\d+)?\s*(?::|：)\s*'
    qa_pairs = []
    for start, end in zip(start_indices[:-1], start_indices[1:]):
        this_qa_text = qa_text[start:end].strip()
        try:
            match = re.match(qaffix_affix + r"\s*(.*?)\n*" + aaffix + r"(.*)", this_qa_text, re.DOTALL)
            if match is not None:
                q = match.group(1).strip()
                a = match.group(2).strip()
                assert len(q) > 0 and len(a) > 0
                qa_pairs.append((q, a))
            else:
                non_match_text.append(this_qa_text)
        except Exception as e:
            non_match_text.append(this_qa_text)
            continue
    items[str(res_item['id'][9:])]['qa'] = qa_pairs
    if len(qa_pairs) == 5:
        non_error_docs.append(res_item)

Processing QA response: 614444it [02:55, 3491.68it/s]


In [24]:
print(items[list(items.keys())[0]])

{'docid': 'sccj1979@@42/2/42_2_121', 'title': 'Makeup Inhibits Reduction of Reactive Oxygen Scavenging Enzyme Activity Induced by Mental Stress', 'abstract': 'Recent studies showed that makeup reduces mental stress, indicated by not only psychological but also physiological values. In this study, we examined the relationship between mental stress and the activity of reactive oxygen scavenging enzymes such as superoxide dismutase (SOD) and catalase (CAT), and the effect of the mental stress reduction by makeup on the activity of reactive oxygen scavenging enzymes. In experiment 1, we measured the concentration of cortisol and activity of SOD in saliva after the addition of mental stress. In this result, the concentration of cortisol increased and the activity of SOD decreased significantly. As we examined the effect of makeup following the mental stress addition in experiment 2, the concentration of cortisol decreased and the activity of SOD and CAT increased. Moreover, we showed a decr

In [25]:
len(non_match_text), len(non_match_doc), len(less_5_qa), len(non_error_docs)

(21, 0, 5, 614439)

In [26]:
from utils import dump_jsonl


# response_fn = os.path.join(DATA_ROOT, "response", "en_pair", "qa-gen.response.s0e620000.jsonl")
new_response_fn = os.path.join(DATA_ROOT, "response", "en_pair", "qa-gen.response.s0e620000.jsonl.filtered")
dump_jsonl(non_error_docs, new_response_fn)

In [5]:
i = 1
print(error_ta[i][0])
print(error_ta[i][1])

Research Progress on Cytoplasmic Male Sterility in Rice
Discoveries in cytoplasmic male sterility have garnered extensive attention from plant breeders. The mechanisms underlying its development are closely linked to the theory of cytoplasmic inheritance. In hybrid rice production, sterile lines play a crucial role in determining the output and planting area of hybrid rice. In fertility restoration research, the primary focus has been on investigating the restoring capability of fertility restoration genes over sterility genes. Previous studies have predominantly focused on investigations involving a single restorer line corresponding to one sterile line. Additionally, some have attempted to use restored hybrid approaches to study the positioning of restoration genes. Different studies have yielded varying conclusions based on the types of interactions and approaches employed. This review provides an overview of the genetics and molecular tagging of CMS types, including the 野败型 (yěbàix

In [8]:
print(error_docs[20])

好，我现在需要将用户提供的中文论文标题和摘要翻译成英文。首先，我会仔细阅读标题和摘要，理解其主要内容和关键术语。

标题是“太湖底泥水华蓝藻复苏的模拟”，这主要涉及太湖、底泥、蓝藻复苏以及模拟研究。我需要确保这些术语在英文中准确无误。"太湖"直接翻译为“Lake Taihu”，“底泥”是“sediment”，“水华蓝藻”应译为“cyanobacteria blooms”，最后“复苏的模拟”可以翻译为“Simulation of Resurgence”。综合起来，标题可以译为“Simulation of Cyanobacteria Blooms from Lake Taihu Sediments”或者更简洁一点，“Simulation Study on the Resurgence of Cyanobacteria Blooms in Lake Taihu Sediments”。

接下来是摘要部分，内容比较详细，涉及实验方法、结果和结论。我需要确保专业术语的准确性，如“光照升温培养”译为“light and temperature incubation”，“色素含量”译为“pigment content”，“藻群体”译为“algal populations”。

在翻译过程中，要保持原意不变，同时确保语言流畅、专业。例如，“在室内模拟条件下”可以译为“under laboratory simulation conditions”，“与环境温度变化密切相关”则是“closely related to environmental temperature changes”。

此外，数字和单位要准确无误，比如“7.2-7.8 μm”保持不变，温度范围如“18-20℃”也要准确翻译。

最后，我会按照用户提供的结构化格式，确保标题和摘要分开，标点符号正确，内容完整。

总结一下，我会逐句翻译，确保每个术语和关键点都准确传达，并且整体读起来专业流畅。
</think>

## **Simulation Study on the Resurgence of Cyanobacteria Blooms in Lake Taihu Sediments**

### **Abstract**

In this experiment, sediment and overlying wat